In [ ]:
import tkinter as tk
from tkinter import messagebox
import sqlite3

conn = sqlite3.connect('budget5.db')
cur = conn.cursor()

cur.execute("DROP TABLE IF EXISTS expenses")
cur.execute("DROP TABLE IF EXISTS Salary_Budget")
cur.execute("DROP TABLE IF EXISTS Month")

cur.execute('''CREATE TABLE IF NOT EXISTS Month(id integer primary key, month_name text, year integer)''')
cur.execute('''CREATE TABLE IF NOT EXISTS Salary_Budget(id integer primary key, Salary_Amount real, Budget_Amount real, month_id Integer,
FOREIGN KEY (month_id) REFERENCES Month(id))''')
cur.execute('''CREATE TABLE IF NOT EXISTS expenses(id integer primary key, expense_name text, amount real, month_id Integer,
FOREIGN KEY (month_id) REFERENCES Month(id))''')

def add_month_year():
    id  = int(id_entry.get())
    year = int(year_entry.get())
    month = month_entry.get()
    cur.execute("INSERT INTO Month (id,month_name,year) VALUES (?,?, ?)", (id,month, year))
    conn.commit()
    messagebox.showinfo("Success", "Month Year added successfully!")

def add_budget_salary():
    id = int(id_sal_entry.get())
    salary = float(salary_entry.get())
    budget = float(budget_entry.get())
    month_id  = int(id_entry.get())
    cur.execute("INSERT INTO Salary_Budget (id,Salary_Amount,Budget_Amount,month_id) VALUES (?,?,?,?)", (id,salary, budget,month_id))
    conn.commit()
    messagebox.showinfo("Success", "Salary and Budget added successfully!") 

def add_expenses():
    id = int(id_exp_entry.get())
    expense_name = expense_name_entry.get()
    expense_amount = float(expense_amount_entry.get())
    month_id  = int(id_entry.get())
    cur.execute("INSERT INTO expenses (id,expense_name,amount,month_id) VALUES (?,?,?,?)", (id,expense_name, expense_amount,month_id))
    conn.commit()
    expenses_text.insert(tk.END, f"{id}:   {expense_name}: Rs {expense_amount:.2f}\n")
    messagebox.showinfo("Success", "Expense added successfully!") 
    update_total_expenses()
    id_exp_entry.delete(0,tk.END)
    expense_name_entry.delete(0, tk.END)
    expense_amount_entry.delete(0,tk.END)
    
def delete_expenses():
    selected = expenses_text.curselection()
    
    if selected:
        selected_value = expenses_text.get(selected[0])
        value_until_colon = selected_value.split(":")[0]
        value_as_int = int(value_until_colon)
        index = selected[0]
        expenses_text.delete(index)
        cur.execute("DELETE FROM expenses WHERE Id=?", (value_as_int,))
        conn.commit()       
        messagebox.showinfo("Success", "Expense deleted successfully!")
        update_total_expenses()
    else:
        messagebox.showerror("Expense Not Found", "No expense selected to delete.")

def update_expenses():
    id   = id_upd_exp_entry.get()
    name = update_name_entry.get()
    new_amount = float(update_amount_entry.get())
    found = False
    for index in range(expenses_text.size()):
        item = expenses_text.get(index)
        if id in item:
            expenses_text.delete(index)
            expenses_text.insert(index, f"{id}:   {name}: Rs {new_amount:.2f}\n")
            found = True
            update_total_expenses()
            break
    if found:
            update_query = "UPDATE expenses SET "
            update_data = []
            update_query+="expense_name=?,"
            update_data.append(name)
            update_query+="amount=?,"
            update_data.append(new_amount)
            update_query = update_query.rstrip(',') + " WHERE id=?"
            update_data.append(id)
            cur.execute(update_query,tuple(update_data))
            conn.commit()
            messagebox.showinfo("Expense Updated", "Expense updated successfully.")
    else:
        messagebox.showerror("Expense Not Found", "Expense not found.")


def calculate_savings():
    try:
        salary = float(salary_entry.get())
        budget = float(budget_entry.get())
        total_expenses = sum(float(expense.split(": Rs ")[1]) for expense in expenses_text.get(0, tk.END))
        savings = salary - total_expenses
        savings_label.config(text=f"Savings: Rs {savings:.2f}")
        salary_display_label.config(text=f"Salary: Rs {salary:.2f}")
        budget_display_label.config(text=f"Budget: Rs {budget:.2f}")
        total_expenses_display_label.config(text=f"Total Expenses: Rs {total_expenses:.2f}")

        percentage_saved = (savings / salary) * 100
        percentage_saved_label.config(text=f"Percentage Saved: {percentage_saved:.2f}%")

        budget = float(budget_entry.get())
        if total_expenses > budget:
            saved_more_less_label.config(text="You have spent more than your budget.")
        elif total_expenses < budget:
            saved_more_less_label.config(text="You have spent less than your budget.")
        else:
            saved_more_less_label.config(text="You have spent exactly your budget.")

    except ValueError:
        messagebox.showerror("Error", "Please enter valid data.")

def update_total_expenses():
    total = sum(float(expense.split(": Rs ")[1]) for expense in expenses_text.get(0, tk.END))
    total_expenses_label.config(text=f"Total Expenses: Rs {total:.2f}")

root = tk.Tk()
root.title("SMART BUDGET TRACKER MANAGEMENT SYSTEM")

# Frame for Month Details
month_year_details_frame = tk.LabelFrame(root, text="Month Year Details", padx=5, pady=10)
month_year_details_frame.grid(row=0, column=0, padx=20, pady=10, sticky="nsew")

# Id Entry Section
id_label = tk.Label(month_year_details_frame, text="Id:", font=("Arial", 14))
id_label.grid(row=0, column=0, sticky="w", padx=5, pady=5)
id_entry = tk.Entry(month_year_details_frame, font=("Arial", 14))
id_entry.grid(row=1, column=0, padx=5, pady=5)

# Year Entry Section
year_label = tk.Label(month_year_details_frame, text="Year:", font=("Arial", 14))
year_label.grid(row=2, column=0, sticky="w", padx=5, pady=5)
year_entry = tk.Entry(month_year_details_frame, font=("Arial", 14))
year_entry.grid(row=3, column=0, padx=5, pady=5)

# Month Entry Section
month_label = tk.Label(month_year_details_frame, text="Month:", font=("Arial", 14))
month_label.grid(row=4, column=0, sticky="w", padx=5, pady=5)
month_entry = tk.Entry(month_year_details_frame, font=("Arial", 14))
month_entry.grid(row=5, column=0, padx=5, pady=5)

# Button to Display Details
add_button = tk.Button(month_year_details_frame, text="Add Details", command=add_month_year, font=("Arial", 14))
add_button.grid(row=6, columnspan=6, padx=8, pady=10)


# Frame for Salary Budget Details
salary_budget_details_frame = tk.LabelFrame(root, text="Monthly Salary Budget Details", padx=5, pady=5)
salary_budget_details_frame.grid(row=0, column=1, padx=20, pady=5, sticky="nsew")

# Id Entry Section
id_label = tk.Label(salary_budget_details_frame, text="Id:", font=("Arial", 14))
id_label.grid(row=0, column=0, sticky="w", padx=5, pady=5)
id_sal_entry = tk.Entry(salary_budget_details_frame, font=("Arial", 14))
id_sal_entry.grid(row=1, column=0, padx=5, pady=5)

# Salary Entry Section
salary_label = tk.Label(salary_budget_details_frame, text="Salary:", font=("Arial", 14))
salary_label.grid(row=2, column=0, sticky="w", padx=5, pady=5)
salary_entry = tk.Entry(salary_budget_details_frame, font=("Arial", 14))
salary_entry.grid(row=3, column=0, padx=5, pady=5)

# Monthly Budget Entry Section
budget_label = tk.Label(salary_budget_details_frame, text="Budget:", font=("Arial", 14))
budget_label.grid(row=4, column=0, sticky="w", padx=5, pady=5)
budget_entry = tk.Entry(salary_budget_details_frame, font=("Arial", 14))
budget_entry.grid(row=5, column=0, padx=5, pady=5)

# Button to Display Details
add_button = tk.Button(salary_budget_details_frame, text="Add Details", command=add_budget_salary, font=("Arial", 14))
add_button.grid(row=6, columnspan=2, padx=8, pady=10)

# Label to Display Details
details_label = tk.Label(salary_budget_details_frame, text="", font=("Arial", 14))
details_label.grid(row=2, columnspan=2, padx=10, pady=5)


# Frame for Add and Delete Expense
add_delete_frame = tk.LabelFrame(root, text="Add and Delete Expense", padx=5, pady=8)
add_delete_frame.grid(row=1, column=0, padx=20, pady=5, sticky="nsew")

# Id Entry Section
id_label = tk.Label(add_delete_frame, text="Id:", font=("Arial", 14))
id_label.grid(row=0, column=0, sticky="w", padx=5, pady=5)
id_exp_entry = tk.Entry(add_delete_frame, font=("Arial", 14))
id_exp_entry.grid(row=1, column=0, padx=5, pady=5)

# Expense Entry Section
expense_name_label = tk.Label(add_delete_frame, text="Expense Name:", font=("Arial", 14))
expense_name_label.grid(row=2, column=0, sticky="w", padx=5, pady=5)
expense_name_entry = tk.Entry(add_delete_frame, font=("Arial", 14))
expense_name_entry.grid(row=3, column=0, padx=5, pady=5)

expense_amount_label = tk.Label(add_delete_frame, text="Amount:", font=("Arial", 14))
expense_amount_label.grid(row=4, column=0, sticky="w", padx=5, pady=5)
expense_amount_entry = tk.Entry(add_delete_frame, font=("Arial", 14))
expense_amount_entry.grid(row=5, column=0, padx=5, pady=5)

# Buttons for Add and Delete Expense
add_expense_button = tk.Button(add_delete_frame, text="Add Expense", command=add_expenses, font=("Arial", 14))
add_expense_button.grid(row=6, columnspan=2, padx=8, pady=10)

delete_expense_button = tk.Button(add_delete_frame, text="Delete Expense", command=delete_expenses, font=("Arial", 14))
delete_expense_button.grid(row=7, columnspan=10, padx=8, pady=10)

# Frame for Update Expense
update_frame = tk.LabelFrame(root, text="Update Expense", padx=5, pady=5)
update_frame.grid(row=1, column=1, padx=20, pady=5, sticky="nsew")

# Id Entry Section
id_label = tk.Label(update_frame, text="Id:", font=("Arial", 14))
id_label.grid(row=0, column=0, sticky="w", padx=5, pady=5)
id_upd_exp_entry = tk.Entry(update_frame, font=("Arial", 14))
id_upd_exp_entry.grid(row=1, column=0, padx=5, pady=5)

# Update Expense Entry Section
update_name_label = tk.Label(update_frame, text="Expense Name:", font=("Arial", 14))
update_name_label.grid(row=2, column=0, sticky="w", padx=5, pady=5)
update_name_entry = tk.Entry(update_frame, font=("Arial", 14))
update_name_entry.grid(row=3, column=0, padx=5, pady=5)

update_amount_label = tk.Label(update_frame, text="Amount:", font=("Arial", 14))
update_amount_label.grid(row=4, column=0, sticky="w", padx=5, pady=5)
update_amount_entry = tk.Entry(update_frame, font=("Arial", 14))
update_amount_entry.grid(row=5, column=0, padx=5, pady=5)

# Update Expense Button
update_expense_button = tk.Button(update_frame, text="Update Expense", command=update_expenses, font=("Arial", 14))
update_expense_button.grid(row=6, columnspan=2, padx=10, pady=10)

# Frame for Expense Listbox
expenses_list_frame = tk.Frame(root)
expenses_list_frame.grid(row=0, column=2, padx=10, pady=10, rowspan=3, sticky="nsew")

# Expenses Listbox
expenses_text_label = tk.Label(expenses_list_frame, text="Expenses:",font=("Arial", 14))
expenses_text_label.grid(row=0, column=0, padx=10, pady=5, sticky="w")

expenses_text = tk.Listbox(expenses_list_frame, width=40, height=28, font=("Arial", 14))
expenses_text.grid(row=1, column=0, padx=10, pady=5, sticky="nsew")


# Label for total expenses
total_expenses_label = tk.Label(root, text="Total Expenses: Rs 0.00", font=("Arial", 14))
total_expenses_label.grid(row=2, column=2, padx=10, pady=5, sticky="w")

# Frame for Savings Calculation
savings_frame = tk.LabelFrame(root, text="Savings", padx=10, pady=10)
savings_frame.grid(row=0, column=3, padx=10, pady=10, sticky="ne")

# Button to Calculate Savings
calculate_savings_button = tk.Button(savings_frame, text="Calculate Savings", command=calculate_savings, font=("Arial", 14))
calculate_savings_button.grid(row=0, columnspan=2, padx=10, pady=10)

# Label to display Salary
salary_display_label = tk.Label(savings_frame, text="Salary: Rs 0.00", font=("Arial", 14))
salary_display_label.grid(row=1, columnspan=2, padx=10, pady=5)

# Label to display Budget
budget_display_label = tk.Label(savings_frame, text="Budget: Rs 0.00", font=("Arial", 14))
budget_display_label.grid(row=2, columnspan=2, padx=10, pady=5)

# Label to display Total Expenditure
total_expenses_display_label = tk.Label(savings_frame, text="Total Expenses: Rs 0.00", font=("Arial", 14))
total_expenses_display_label.grid(row=3, columnspan=2, padx=10, pady=5)

# Label to display Savings
savings_label = tk.Label(savings_frame, text="Savings: Rs 0.00", font=("Arial", 14))
savings_label.grid(row=4, columnspan=2, padx=10, pady=5)

# Label for Percentage Saved
percentage_saved_label = tk.Label(savings_frame, text="Percentage Saved: 0.00%", font=("Arial", 14))
percentage_saved_label.grid(row=5, columnspan=2, padx=10, pady=5)

# Label for Saved More/Less
saved_more_less_label = tk.Label(savings_frame, text="", font=("Arial", 14))
saved_more_less_label.grid(row=6, columnspan=2, padx=10, pady=5)


root.mainloop()
